### Spiral Matrix

#### Problem

Given an `m x n` matrix, such as:
```
1  2  3  4  5
6  7  8  9  10
11 12 13 14 15
16 17 18 19 20
```

write a function that returns the entries of the matrix in a spiral fashion starting at the top-left.

For the example above, result should be:

```1 2 3 4 5 10 15 20 19 18 17 16 11 6 7 8 9 14 13 12```

We'll provide three different solutions:
1. brute force method
2. slightly refined brute force method, using recursion
3. solution that uses mutation, cyclically (is that a word??) removing rows, and columns until the matrix has been emptied.

There are pros and cons to each implementation:
- Brute force method is not elegant, and code is hard to follow.
- Recursive approach is easier to understand because each recursion step deals with one spiral only - but problem is recursion depth is limited and uses a lot of memory to maintain all the stack frames.
- Both brute force and recursive methods assume square matrices - I tried generalizing it to non-square matrices, but the code just got horrendous (there's probably a better more readable approach, but that eludes me for now...)
- The elimination solution is by far the easiest to understand, and, as an added bonus, it handles with rectangular matrices. It will not deal properly with ragged matrices (so we do require that each row contain the same number of elements).
- The downside of the elimination method is it requires duplication of the data - so more memory overhead than a pure iterative solution like #1.

#### Solution 1

This solution assumes a non-empty square matrix (1x1 is ok), and uses a set of repeating directions used to travel throughout the structure.

We can think of the matrix, as having rows and columns set up as follows, and moving directionally within the matrix:

```
             columns
        ------------->
        |
 rows   |
        |
        v
```

Directions need to encode horizontal and vertical directions. 

We'll use a tuple `(row, column)`, where each elements of the tuple can be `1`, `-1` or `0`, indicating incrementing, decrementing or leaving constant respectively.

To complete one spiral, we need to perform the following "steps" from each starting point:
0. yield starting cell
1. move and yield right until "end" of row, so direction is `(0, 1)`
2. move and yield down until "end" of column, so direction is `(1, 0)`
3. move and yield left until "beginning" of row, so direction is `(0, -1)`
4. move and yield up until "beginning" of column, so direction is `(-1, 0)`

Then we keep repeating these steps until we're done iterating through all the elements.

So, we have a cycle of "movements" that make up one cycle, and just needs to be repeated.

The only thing left is how we determine how many movements to make in each direction.

If we consider an `nxn` matrix, we start at the first cell, then move `n-1` steps right, `n-1` steps down, `n-1` steps left and `n-2` steps up. We then need to repeat this operation for the "remaining" matrix, which will have `2` less rows and `2` less columns.

First solution is a non-recursive approach.

In [1]:
from itertools import chain, repeat

def iter_matrix(m):
    dirs = [(0,1), (1,0), (0, -1), (-1, 0)]
    row = 0
    col = -1
    n = len(m[0])
    while n >= 1:
        col += 1
        yield m[row][col]
        if n == 1:
            # this was a 1x1 matrix, so done
            return
        moves = chain(repeat(dirs[0], n-1), repeat(dirs[1], n-1), repeat(dirs[2], n-1), repeat(dirs[3], n-2))
        # could also write this as follows, but harder to understand and not much less code:
        # moves = chain.from_iterable(repeat(dirs[i], n-2 if i == 3 else n-1) for i in range(4))
        for move in moves:
            row, col = row + move[0], col + move[1]
            yield m[row][col]
        n -= 2

#### Solution 2

So now, can make this into a recursive call as well, where each call prints one "spiral", and calls itself for the remaining elements. At each iteration we need to know the starting point in the overall matrix, and the matrix size (which decreases by 2 each iteration).

This solution is (in my mind) clearer, but Python has a recursion depth limit (which can be increased, but probably unwise). Still this solution will work for matrices sized up to twice (more or less, depending on even or odd size) that depth limit:

In [2]:
import sys

print('Current recursion limit:', sys.getrecursionlimit())

Current recursion limit: 3000


In [3]:
from itertools import chain, repeat

def recurse_iter_matrix(m, row=0, col=0, effective_size=None):
    # use full matrix by default
    n = effective_size if effective_size is not None else len(m)

    dirs = [(0,1), (1,0), (0, -1), (-1, 0)]
    
    if n <= 0:
        # empty matrix, nothing to do - exit recursion
        return
    
    yield m[row][col]
    
    if n == 1:
        # 1x1 matrix, so done after yielding first (and only) cell
        return
    
    moves = list(chain(repeat(dirs[0], n-1), repeat(dirs[1], n-1), repeat(dirs[2], n-1), repeat(dirs[3], n-2)))
    for move in moves:
        row, col = row + move[0], col + move[1]
        yield m[row][col]
    
    yield from recurse_iter_matrix(m, row, col+1, n-2)

#### Examples using solutions 1 and 2

Requires matrices to be square and non-empty.

In [4]:
samples = [
    [[1,2,3,4,5], [6,7,8,9,10], [11,12,13,14,15],[16,17,18,19,20],[21,22,23,24,25]],
    [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]],
    [[1,2,3],[4,5,6],[7,8,9]],
    [[1,2], [3,4]],
    [[1]]
]

In [5]:
from pprint import pprint

for sample in samples:
    print('-' * 20)
    print('Input:')
    pprint(sample, width=len(sample) * 5)
    print('\nIteration:')
    print(list(iter_matrix(sample)))
    print('\nRecursive:')
    print(list(recurse_iter_matrix(sample)), end='\n\n')

--------------------
Input:
[[1, 2, 3, 4, 5],
 [6, 7, 8, 9, 10],
 [11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20],
 [21, 22, 23, 24, 25]]

Iteration:
[1, 2, 3, 4, 5, 10, 15, 20, 25, 24, 23, 22, 21, 16, 11, 6, 7, 8, 9, 14, 19, 18, 17, 12, 13]

Recursive:
[1, 2, 3, 4, 5, 10, 15, 20, 25, 24, 23, 22, 21, 16, 11, 6, 7, 8, 9, 14, 19, 18, 17, 12, 13]

--------------------
Input:
[[1, 2, 3, 4],
 [5, 6, 7, 8],
 [9, 10, 11, 12],
 [13, 14, 15, 16]]

Iteration:
[1, 2, 3, 4, 8, 12, 16, 15, 14, 13, 9, 5, 6, 7, 11, 10]

Recursive:
[1, 2, 3, 4, 8, 12, 16, 15, 14, 13, 9, 5, 6, 7, 11, 10]

--------------------
Input:
[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9]]

Iteration:
[1, 2, 3, 6, 9, 8, 7, 4, 5]

Recursive:
[1, 2, 3, 6, 9, 8, 7, 4, 5]

--------------------
Input:
[[1, 2],
 [3, 4]]

Iteration:
[1, 2, 4, 3]

Recursive:
[1, 2, 4, 3]

--------------------
Input:
[[1]]

Iteration:
[1]

Recursive:
[1]



#### Solution 3

This approach keeps trimming rows (top and bottom), and columns (left and right) until the matrix is empty.

We establish a few helper functions first.

In [6]:
def eliminate_row(m, *, first=True, reverse=False):
    """Trims first or last row, and returns iterator in original/reversed order. 
    No assumption on matrix size/shape is made. 
    
    Note: this not only returns the eliminated row, but also **mutates** the matrix m!
    
    :param m: 2D matrix (list of lists), will be mutated
    :param first: True to eliminate first row, False to eliminate last row
    :param reverse: return eliminated row in natural order (reverse=False), 
            or reversed order (reverse=True)
    :return: returns an iterator
    """
    row = m.pop(0 if first else -1)
    return list(reversed(row)) if reverse else row

In [7]:
def eliminate_column(m, *, first=True, reverse=False):
    """Trims first or last column, and returns iterator in original/reversed order. 
    No assumption on matrix size/shape is made. 
    
    Note: this not only returns the eliminated column, but also **mutates** the matrix m!
    
    param m: 2D matrix (list of lists), will be mutated
    param first: True to eliminate first column, False to eliminate last column
    param reverse: return eliminated column in natural order (reverse=False), 
    or reversed order (reverse=True)
    """
    
    column = [row.pop(0 if first else -1) for row in m]
    return list(reversed(column)) if reverse else column

We're now ready to spiral through the matrix. No assumption need be made here about the shape/size of the matrix, other than it not be ragged (i.e. all rows have an equal number of elements).

Also, we won't make the assumption that the input matrix is mutable, just an iterable of iterables. 

In [8]:
from functools import partial
from itertools import cycle

def eliminate_iter_matrix(m):
    """Generator that yields items in spiral"""
    
    # need a mutable list of lists for internal processing
    # in any case, we also want a copy of the structure since we'll be mutating
    # it, and don't want to mess with caller's data
    # deepcopy won't be enough since we also want to make sure we have mutable data structures
    m = [list(row) for row in m]
    
    # we make these eliminations in this order, and cycle until matrix is empty
    moves = [
        partial(eliminate_row, first=True, reverse=False),
        partial(eliminate_column, first=False, reverse=False),
        partial(eliminate_row, first=False, reverse=True),
        partial(eliminate_column, first=True, reverse=True)
    ]
    
    cycle_moves = cycle(moves)
    
    while m:
        move = next(cycle_moves)
        yield from move(m)

#### Examples

We'll try the same square matrices as before, and include results from all methods - (hopefully they all match!)

Requires matrices to be rectangular and non-empty.

In [9]:
from pprint import pprint

for sample in samples:
    print('-' * 20)
    print('Input:')
    pprint(sample, width=len(sample) * 5)
    print('\nIteration:')
    print(list(iter_matrix(sample)))
    print('\nRecursive:')
    print(list(recurse_iter_matrix(sample)))
    print('\nElimination:')
    print(list(eliminate_iter_matrix(sample)), end='\n\n')

--------------------
Input:
[[1, 2, 3, 4, 5],
 [6, 7, 8, 9, 10],
 [11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20],
 [21, 22, 23, 24, 25]]

Iteration:
[1, 2, 3, 4, 5, 10, 15, 20, 25, 24, 23, 22, 21, 16, 11, 6, 7, 8, 9, 14, 19, 18, 17, 12, 13]

Recursive:
[1, 2, 3, 4, 5, 10, 15, 20, 25, 24, 23, 22, 21, 16, 11, 6, 7, 8, 9, 14, 19, 18, 17, 12, 13]

Elimination:
[1, 2, 3, 4, 5, 10, 15, 20, 25, 24, 23, 22, 21, 16, 11, 6, 7, 8, 9, 14, 19, 18, 17, 12, 13]

--------------------
Input:
[[1, 2, 3, 4],
 [5, 6, 7, 8],
 [9, 10, 11, 12],
 [13, 14, 15, 16]]

Iteration:
[1, 2, 3, 4, 8, 12, 16, 15, 14, 13, 9, 5, 6, 7, 11, 10]

Recursive:
[1, 2, 3, 4, 8, 12, 16, 15, 14, 13, 9, 5, 6, 7, 11, 10]

Elimination:
[1, 2, 3, 4, 8, 12, 16, 15, 14, 13, 9, 5, 6, 7, 11, 10]

--------------------
Input:
[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9]]

Iteration:
[1, 2, 3, 6, 9, 8, 7, 4, 5]

Recursive:
[1, 2, 3, 6, 9, 8, 7, 4, 5]

Elimination:
[1, 2, 3, 6, 9, 8, 7, 4, 5]

--------------------
Input:
[[1, 2],
 [3, 4]]

Iteration:
[1, 

But this solution is not limited to square matrices. Here are some examples:

In [10]:
from pprint import pprint
samples = [
    [[1, 2, 3], [4, 5, 6]],
    [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]],
    [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],
    [[1, 2]],
    [[1], [2]],
    [[1]]
]

for sample in samples:
    print('-' * 20)
    print('Input:')
    pprint(sample, width=max(len(row) for row in sample) * 5)
    print()
    print(list(eliminate_iter_matrix(sample)), end='\n\n')

--------------------
Input:
[[1, 2, 3],
 [4, 5, 6]]

[1, 2, 3, 6, 5, 4]

--------------------
Input:
[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12]]

[1, 2, 3, 6, 9, 12, 11, 10, 7, 4, 5, 8]

--------------------
Input:
[[1, 2, 3, 4],
 [5, 6, 7, 8],
 [9, 10, 11, 12]]

[1, 2, 3, 4, 8, 12, 11, 10, 9, 5, 6, 7]

--------------------
Input:
[[1, 2]]

[1, 2]

--------------------
Input:
[[1],
 [2]]

[1, 2]

--------------------
Input:
[[1]]

[1]

